# small course on the inverse theory

jan 21-23 2026 @ Lyon with IsaVeroSteph 

inspired by the memo prepared by Isa Yael, you can look at the symbolic and concrete way of looking at the normal equations

and how the gradient direction, hessian and fréchet derivatives are linked in the linearised system

we can even put some arbitrary non-linear analytical expressions for u(m) 

which can be useful for my class next year :-)


In [1]:
using Pkg

cd(@__DIR__)
Pkg.activate("../")

  Activating project at `~/Documents/Github/flexOPT`


In [2]:
using Symbolics
include("../src/commonBatchs.jl")
using .commonBatchs

problem setting

In [3]:
# dimensions
modelDimension = 2
dataDimension  = 3

# models (nₑ distribution in the Earth)
@variables m[1:modelDimension] # a model
@variables mTrue[1:modelDimension] # the ground truth
@variables m⁰[1:modelDimension] # our initial guess (PREM or 3D initial model)
@variables δm[1:modelDimension] # an arbitrary model perturbation from m₀ : m = m₀+δm
@variables Δm[1:modelDimension]
δm = collect(δm)
Δm =[mTrue[i] - m⁰[i] for i ∈ 1:modelDimension] # model perturbation required to have the goround truth
#δm =[m[i] - m⁰[i] for i ∈ 1:modelDimension]
# data (predicted and observed), since we mimic 'symbolic partial derivations' by multiplications, u=G(m) stays only in our heart
# maybe WHATEVER!!!
@variables u(m...)[1:dataDimension]
@variables d[1:dataDimension] 
@variables Δd[1:dataDimension] # data - u(m⁰) but I will substitute at the end

δd = [d[i] - u[i] for i ∈ 1:dataDimension] # data residual for synthetic data predicted for an arbitrary model m = m₀+δm

# partials (in a wild symbolic way), but all the ∇s are linear so let's be super super simple (I don't really use the functionalities of )

#@variables ∂m[1:modelDimension]
#@variables ∂δm[1:modelDimension]
#@variables ∂m⁰[1:modelDimension]
∂m = [Differential(m[i]) for i ∈ 1:modelDimension]
∂m⁰ = [Differential(m⁰[i]) for i ∈ 1:modelDimension]
∂δm = [Differential(δm[i]) for i ∈ 1:modelDimension]

2-element Vector{Differential}:
 Differential(δm[1])
 Differential(δm[2])

In [4]:
#cost function for F(m)
F = expand(1//2*sum(δd[i]^2 for i in 1:dataDimension))

(1//2)*(d[1]^2) - d[1]*(u(m[1], m[2]))[1] + (1//2)*(d[2]^2) - d[2]*(u(m[1], m[2]))[2] + (1//2)*(d[3]^2) - d[3]*(u(m[1], m[2]))[3] + (1//2)*((u(m[1], m[2]))[1]^2) + (1//2)*((u(m[1], m[2]))[2]^2) + (1//2)*((u(m[1], m[2]))[3]^2)

Let's develop a Taylor expansion 

In [6]:
# first derivative = gradient direction
∇F = [expand_derivatives(∂m[i](F)) for i ∈ 1:modelDimension] # sorry that ∂ can be found after the mathematical expressions but here it's treated as variables and it's ok to manipulate manually

2-element Vector{Num}:
 -d[1]*Differential(m[1])((u(m[1], m[2]))[1]) - d[2]*Differential(m[1])((u(m[1], m[2]))[2]) - d[3]*Differential(m[1])((u(m[1], m[2]))[3]) + (u(m[1], m[2]))[1]*Differential(m[1])((u(m[1], m[2]))[1]) + (u(m[1], m[2]))[2]*Differential(m[1])((u(m[1], m[2]))[2]) + (u(m[1], m[2]))[3]*Differential(m[1])((u(m[1], m[2]))[3])
 -d[1]*Differential(m[2])((u(m[1], m[2]))[1]) - d[2]*Differential(m[2])((u(m[1], m[2]))[2]) - d[3]*Differential(m[2])((u(m[1], m[2]))[3]) + (u(m[1], m[2]))[1]*Differential(m[2])((u(m[1], m[2]))[1]) + (u(m[1], m[2]))[2]*Differential(m[2])((u(m[1], m[2]))[2]) + (u(m[1], m[2]))[3]*Differential(m[2])((u(m[1], m[2]))[3])

In [7]:
∇∇F = [expand_derivatives(∂m[j](∇F[i])) for i ∈ 1:modelDimension, j ∈ 1:modelDimension ] # second derivatives

2×2 Matrix{Num}:
                                                                                                                -d[1]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[1])) - d[2]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[2])) - d[3]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[3])) + (u(m[1], m[2]))[1]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[1])) + (u(m[1], m[2]))[2]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[2])) + (u(m[1], m[2]))[3]*Differential(m[1])(Differential(m[1])((u(m[1], m[2]))[3])) + Differential(m[1])((u(m[1], m[2]))[1])^2 + Differential(m[1])((u(m[1], m[2]))[2])^2 + Differential(m[1])((u(m[1], m[2]))[3])^2  …  -d[1]*Differential(m[2])(Differential(m[1])((u(m[1], m[2]))[1])) - d[2]*Differential(m[2])(Differential(m[1])((u(m[1], m[2]))[2])) - d[3]*Differential(m[2])(Differential(m[1])((u(m[1], m[2]))[3])) + (u(m[1], m[2]))[1]*Differential(m[2])(Differential(m[1])((u(m[1], m[2]))[1])) + (u(m[1], m[2]))[2]*Dif

In [29]:

# if you don't do this, just comment it to see what's happening at the end
ignore_second_derivatives = Dict(∂m[i](∂m[j](u[k]))=>0 for i ∈ 1:modelDimension, j ∈ 1:modelDimension, k ∈ 1:dataDimension) ; 

In [30]:
∇∇F = expand.(substitute.(∇∇F, Ref(ignore_second_derivatives)))

2×2 Matrix{Num}:
                                                                                                                Differential(m[1])((u(m[1], m[2]))[1])^2 + Differential(m[1])((u(m[1], m[2]))[2])^2 + Differential(m[1])((u(m[1], m[2]))[3])^2  …  Differential(m[1])((u(m[1], m[2]))[1])*Differential(m[2])((u(m[1], m[2]))[1]) + Differential(m[1])((u(m[1], m[2]))[2])*Differential(m[2])((u(m[1], m[2]))[2]) + Differential(m[1])((u(m[1], m[2]))[3])*Differential(m[2])((u(m[1], m[2]))[3])
 Differential(m[1])((u(m[1], m[2]))[1])*Differential(m[2])((u(m[1], m[2]))[1]) + Differential(m[1])((u(m[1], m[2]))[2])*Differential(m[2])((u(m[1], m[2]))[2]) + Differential(m[1])((u(m[1], m[2]))[3])*Differential(m[2])((u(m[1], m[2]))[3])                                                                                                                    Differential(m[2])((u(m[1], m[2]))[1])^2 + Differential(m[2])((u(m[1], m[2]))[2])^2 + Differential(m[2])((u(m[1], m[2]))[3])^2

# substitution for $m^0$

In [31]:
dict_for_m⁰ = Dict(m[i] => m⁰[i] for i ∈ 1:modelDimension)
dict_for_∂m⁰ = Dict(∂m[i] => ∂m⁰[i] for i ∈ 1:modelDimension)
dict_for_m⁰= merge(dict_for_m⁰,dict_for_∂m⁰)

Dict{Any, Any} with 4 entries:
  m[1]               => m⁰[1]
  Differential(m[2]) => Differential(m⁰[2])
  Differential(m[1]) => Differential(m⁰[1])
  m[2]               => m⁰[2]

In [32]:
F⁰= substitute(F,dict_for_m⁰)

(1//2)*(d[1]^2) - d[1]*(u(m⁰[1], m⁰[2]))[1] + (1//2)*(d[2]^2) - d[2]*(u(m⁰[1], m⁰[2]))[2] + (1//2)*(d[3]^2) - d[3]*(u(m⁰[1], m⁰[2]))[3] + (1//2)*((u(m⁰[1], m⁰[2]))[1]^2) + (1//2)*((u(m⁰[1], m⁰[2]))[2]^2) + (1//2)*((u(m⁰[1], m⁰[2]))[3]^2)

In [33]:
∇F⁰= expand.(substitute.(∇F,Ref(dict_for_m⁰))) 

2-element Vector{Num}:
 -d[1]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1]) - d[2]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2]) - d[3]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3]) + (u(m⁰[1], m⁰[2]))[1]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1]) + (u(m⁰[1], m⁰[2]))[2]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2]) + (u(m⁰[1], m⁰[2]))[3]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])
 -d[1]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1]) - d[2]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2]) - d[3]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3]) + (u(m⁰[1], m⁰[2]))[1]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1]) + (u(m⁰[1], m⁰[2]))[2]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2]) + (u(m⁰[1], m⁰[2]))[3]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])

In [34]:


∇∇F⁰= expand.(substitute.(∇∇F,Ref(dict_for_m⁰)))

2×2 Matrix{Num}:
                                                                                                                         Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])^2 + Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])^2 + Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])^2  …  Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3]) + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1]) + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])
 Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3]) + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1]) + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])                                                                                                                             Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])^2 + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1

In [35]:
# we want to develop a Taylor expansion 
F_Taylor⁰ = F⁰ + expand(∇F⁰'*δm) + expand(1//2*δm'*∇∇F⁰*δm) # and δm^3 -> ignored

(1//2)*(d[1]^2) - d[1]*(u(m⁰[1], m⁰[2]))[1] + (1//2)*(d[2]^2) - d[2]*(u(m⁰[1], m⁰[2]))[2] + (1//2)*(d[3]^2) - d[3]*(u(m⁰[1], m⁰[2]))[3] + (1//2)*((u(m⁰[1], m⁰[2]))[1]^2) + (1//2)*((u(m⁰[1], m⁰[2]))[2]^2) + (1//2)*((u(m⁰[1], m⁰[2]))[3]^2) - d[1]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*δm[2] - d[1]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])*δm[1] - d[2]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*δm[2] - d[2]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])*δm[1] - d[3]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*δm[2] - d[3]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])*δm[1] + (u(m⁰[1], m⁰[2]))[1]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*δm[2] + (u(m⁰[1], m⁰[2]))[1]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])*δm[1] + (u(m⁰[1], m⁰[2]))[2]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*δm[2] + (u(m⁰[1], m⁰[2]))[2]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])*δm[1] + (u(m⁰[1], m⁰[2]))[3]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*δm[2] + (u(m⁰[1], m⁰[2]))[3]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])*δm[1] + (1//

We are just looking for $\delta m$ which can make $F^0$ has its plateau
\begin{equation}
\nabla_{\delta m}  F = 0
\end{equation}

In [36]:
∇F_Taylor⁰ = [expand_derivatives(expand(∂δm[i](F_Taylor⁰))) for i ∈ 1:modelDimension]

2-element Vector{Num}:
 -d[1]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1]) - d[2]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2]) - d[3]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3]) + (u(m⁰[1], m⁰[2]))[1]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1]) + (u(m⁰[1], m⁰[2]))[2]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2]) + (u(m⁰[1], m⁰[2]))[3]*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3]) + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])*δm[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])*δm[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])*δm[2] + (Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])^2)*δm[1] + (Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])^2)*δm[1] + (Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])^2)*δm[1]
 -d[1]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1]) - d[2]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2]) - d[3]*Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3]) + (u(m⁰[1], m⁰[2]))[1]*Differential(m⁰[2])((u(m⁰[1

In [38]:
dict_introduce_Δd = Dict(d[i]=> u[i]+Δd[i] for i ∈ 1:dataDimension)

Dict{Num, Num} with 3 entries:
  d[3] => (u(m[1], m[2]))[3] + Δd[3]
  d[1] => (u(m[1], m[2]))[1] + Δd[1]
  d[2] => (u(m[1], m[2]))[2] + Δd[2]

In [39]:
∇F_Taylor⁰ = expand.(Symbolics.scalarize(substitute.(∇F_Taylor⁰,Ref(dict_introduce_Δd ))))
∇F_Taylor⁰ = substitute.(∇F_Taylor⁰,Ref(dict_for_m⁰))

2-element Vector{Num}:
 -Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])*Δd[3] - Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])*Δd[1] - Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])*Δd[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])*δm[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])*δm[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])*δm[2] + (Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])^2)*δm[1] + (Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])^2)*δm[1] + (Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])^2)*δm[1]
 -Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*Δd[3] - Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*Δd[1] - Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*Δd[2] + (Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])^2)*δm[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])*δm[1] + (Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])^2)*δm[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))

In [40]:
# relation between the gradient direction and A⁰Δd

∇F⁰ = expand.(substitute.(∇F,Ref(dict_introduce_Δd)))
∇F⁰ = substitute.(∇F⁰,Ref(dict_for_m⁰))


2-element Vector{Num}:
 -Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])*Δd[3] - Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])*Δd[1] - Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])*Δd[2]
 -Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*Δd[3] - Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*Δd[1] - Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*Δd[2]

In [41]:

# check that the gradient direction has - Fréchet derivative times Δd
A⁰=Symbolics.jacobian(∇F⁰,Δd)'

3×2 adjoint(::Matrix{Num}) with eltype Num:
 -Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])  …  -Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])
 -Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])     -Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])
 -Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])     -Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])

In [20]:
hessianContribution = ∇F_Taylor⁰ - ∇F⁰

2-element Vector{Num}:
 Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])*δm[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])*δm[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])*δm[2] + (Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])^2)*δm[1] + (Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])^2)*δm[1] + (Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])^2)*δm[1]
 (Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])^2)*δm[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])*δm[1] + (Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])^2)*δm[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])*δm[1] + (Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])^2)*δm[2] + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])*δm[1]

In [21]:
H = Symbolics.jacobian(hessianContribution,δm)

2×2 Matrix{Num}:
                                                                                                                         Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3])^2 + Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1])^2 + Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])^2  …  Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3]) + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1]) + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])
 Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[3]) + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[1]) + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[2])*Differential(m⁰[1])((u(m⁰[1], m⁰[2]))[2])                                                                                                                             Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[3])^2 + Differential(m⁰[2])((u(m⁰[1], m⁰[2]))[1

In [22]:
H-A⁰'*A⁰

2×2 Matrix{Num}:
 0  0
 0  0

In [42]:
HAHAHAAAA

UndefVarError: UndefVarError: `HAHAHAAAA` not defined in `Main`
Suggestion: check for spelling errors or missing imports.